En el presente archivo se muestra el proceso realizado para la limpieza y transformación de los datasets proporcionados para el proyecto
<hr>

Inicialmente se importan las librerías necesarias

In [138]:
import pandas as pd
import numpy as np

Se crean los dataframes para cada plataforma con los archivos CSV proporcionados.

In [139]:
df_amazon = pd.read_csv(r'./Datasets/amazon_prime_titles-score.csv')
df_disney = pd.read_csv(r'./Datasets/disney_plus_titles-score.csv')
df_hulu = pd.read_csv(r'./Datasets/hulu_titles-score (2).csv')
df_netflix = pd.read_csv(r'./Datasets/netflix_titles-score.csv')

Se crea la columna 'id' concatenando la letra inicial del nombre de la plataforma, con el código de la columna 'show_id'

In [140]:
df_amazon.insert(0, 'id', ('a' + df_amazon['show_id']))
df_disney.insert(0, 'id', ('d' + df_disney['show_id']))
df_hulu.insert(0, 'id', ('h' + df_hulu['show_id']))
df_netflix.insert(0, 'id', ('n' + df_netflix['show_id']))

Se concatenan los cuatro dataframes en un unico dataframe.

In [141]:
df_plataformas = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], axis=0)

Se rellenan los espacios vacíos de la columna 'rating' con la letra 'g' 

In [142]:
df_plataformas['rating'] = df_plataformas['rating'].fillna('g')

Se cambia el formato de la columna 'date_added' para que sea datetime

In [143]:
df_plataformas['date_added'] = pd.to_datetime(df_plataformas['date_added'], infer_datetime_format=True)

Se separa la columna 'duration' en dos columnas ('duration_int' y 'duration_type') para que en la primera queden los números y en la segunda el tipo.

In [144]:
df_plataformas[['duration_int', 'duration_type']] = df_plataformas['duration'].str.split(expand=True)

Para evitar problemas posteriores se rellenan los espacios nulos con 0. Se tomó esta decisión ya que se observó que las columnas donde había valores nulos no afectaban a las consultas solicitadas y el dejar los registros como nulos sí podría afectar posteriormente.

In [146]:
df_plataformas = df_plataformas.fillna('0')

Se cambia el tipo de dato de la columna 'duration_int' para que sea integer.

In [148]:
df_plataformas['duration_int'] = df_plataformas['duration_int'].astype('int64')

Se normaliza 'Seasons' a 'Season'

In [149]:
df_plataformas['duration_type'].replace(to_replace = 'Seasons', value='Season', inplace=True)

Se define una función para cambiar a minúsculas todos los datos alojados en columnas de tipo object

In [150]:
def minusculas(dataframe):
    columnas = dataframe.select_dtypes(include=object).columns
    for i in columnas:
        dataframe[i] = dataframe[i].str.lower()
    return dataframe.head()

Se utiliza la función minúsculas para dejar todos los strings en minúscula.

In [151]:
minusculas(df_plataformas)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,score,duration_int,duration_type
0,as1,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,NaN,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...,99,113,min
1,as2,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,NaN,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...,37,110,min
2,as3,s3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,NaN,2017,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...,20,74,min
3,as4,s4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,NaN,2014,g,69 min,documentary,"pink breaks the mold once again, bringing her ...",27,69,min
4,as5,s5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,NaN,1989,g,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...,75,45,min


Se exporta el dataframe, en formato CSV, con las transformaciones necesarias para su consulta.

In [153]:
df_plataformas.to_csv('Plataformas_Unificado.csv', index= False)